<h1 align='center'> 영상처리 프로그래밍 실습 10</h1>

<h6 align='right'> 2025. 6. 5. </h6>

<div class="alert alert-block alert-info">
    
- 파일 이름에서 00000000을 자신의 학번으로, name을 자신의 이름으로 수정하세요.

- 다음 줄에 자신의 이름, 학번, 학과(전공)을 적으세요.

* 이름: 이선재  &nbsp;&nbsp;          학번:  20227123   &nbsp;&nbsp;         학과(전공): 빅데이터
    
</div>

- JupyterLab 문서의 최신 버전은 [JupyterLab Documentation](https://jupyterlab.readthedocs.io/en/stable/index.html#/)을  참고하라

- Markdown은 [Markdown Guide](https://www.markdownguide.org/)를 참고하라.
- [Markdown Cheat Sheet](https://www.markdownguide.org/cheat-sheet/)
- 실험에 사용할 영상은 [영상 데이터베이스](https://drive.google.com/drive/folders/1zbjtkf9nHy9VniuLI4wHilbrN_JBvhYi)에서 다운로드할 것


* 제출 마감: 6월 11일 (수) 오후 10:00까지 최종본을 SmartLEAD제출


<details>
<summary>📝 질문과 답 1 (Sample)</summary>

### 질문 
---


### 답변
---

</details>

In [1]:
import cv2
import matplotlib.pyplot as plt

import numpy as np
print("OpenCV version", cv2.__version__)
print("NumPy version", np.__version__)

OpenCV version 4.11.0
NumPy version 1.26.4


# 문제 1. 
 
다음 조건을 만족하는 함수 perspective_projection_transform(img) 함수를 작성하라.

- OpenCV의 윈도우을 만들고, 입력 영상을 왼쪽에, 입력 영상의 복사본을 오른쪽에 나란히 표시한다
- 왼쪽 영상의 4 각형의 네 점의 위치를 다음 순서대로 마우스로 클릭하면 4 각형을 그린다.
- 왼 꼭짓점에서 반시계 방향으로 지정한다
- 사각형이 완성되면 perspective projection transform된 영상을 오른쪽에 표시한다.
- perspective projection transform 출력 영상의 크기는 입력 영상의 크기와 같다

다음 그림은 'social-media_640.jpg' 파일을 이 프로그램에 적용한 결과 예이다.

![perspective_Tx_Sosical_media](perspective_Tx_Sosical_media.png)

In [26]:
import cv2
import numpy as np

# 마우스 클릭한 점들을 저장할 리스트
points = []

def mouse_callback(event, x, y, flags, param):
    global points, img_copy

    if event == cv2.EVENT_LBUTTONDOWN and len(points) < 4:
        points.append((x, y))
        # 점 찍기
        cv2.circle(img_copy, (x, y), 5, (0, 255, 0), -1)
        if len(points) > 1:
            cv2.line(img_copy, points[-2], points[-1], (255, 0, 0), 2)
        # 마지막 점일 경우 첫점과 연결
        if len(points) == 4:
            cv2.line(img_copy, points[3], points[0], (255, 0, 0), 2)

def perspective_projection_transform(img):
    global points, img_copy

    h, w = img.shape[:2]
    # 왼쪽 원본 + 오른쪽 복사본을 옆으로 붙이기
    display_img = np.hstack((img.copy(), img.copy()))
    img_copy = display_img.copy()

    cv2.namedWindow("Perspective Transform")
    cv2.setMouseCallback("Perspective Transform", mouse_callback)

    while True:
        cv2.imshow("Perspective Transform", img_copy)

        key = cv2.waitKey(1) & 0xFF

        # ESC 키로 종료
        if key == 27:
            break

        # 4점 선택되면 변환 수행
        if len(points) == 4:
            # 변환 대상 점 (사각형)
            dst_pts = np.float32([
            [0, 0],          # 왼쪽 위
            [0, h - 1],      # 왼쪽 아래
            [w - 1, h - 1],  # 오른쪽 아래
            [w - 1, 0]       # 오른쪽 위
        ])

        # perspective transform
        M = cv2.getPerspectiveTransform(np.float32(points), dst_pts)
        warped = cv2.warpPerspective(img, M, (w, h))

        # 오른쪽 영역에 결과 붙이기
        img_copy[:, w:] = warped

        # 변환 완료 후 다시 클릭 못하게 막기
        points.clear()

    cv2.destroyAllWindows()

# 예시 실행
# 순서 왼쪽 위, 오른쪽 위, 오른쪽 아래, 왼쪽 아래
img = cv2.imread('social-media_640.jpg')
perspective_projection_transform(img)


<details>
<summary>📝 질문과 답 1 (기초 구현) - chat gpt 4o</summary>

### 질문 
---
### 문제 1. 
 
다음 조건을 만족하는 함수 perspective_projection_transform(img) 함수를 작성하라.

- OpenCV의 윈도우을 만들고, 입력 영상을 왼쪽에, 입력 영상의 복사본을 오른쪽에 나란히 표시한다
- 왼쪽 영상의 4 각형의 네 점의 위치를 다음 순서대로 마우스로 클릭하면 4 각형을 그린다.
- 사각형이 완성되면 perspective projection transform된 영상을 오른쪽에 표시한다.
- perspective projection transform 출력 영상의 크기는 입력 영상의 크기와 같다

다음 그림은 'social-media_640.jpg' 파일을 이 프로그램에 적용한 결과 예이다.

![perspective_Tx_Sosical_media](perspective_Tx_Sosical_media.png)


### 답변
---
다음은 요구사항을 충족하는 OpenCV 기반의 `perspective_projection_transform(img)` 함수 전체 코드입니다.

이 코드는:

* 입력 이미지와 같은 크기의 오른쪽 복사본을 포함한 창을 표시하고,
* 사용자가 마우스로 왼쪽 이미지에 4점을 클릭하면 사각형을 그리고,
* 이 사각형에 대해 **투시 변환(perspective transform)** 을 수행하여,
* 오른쪽 이미지 영역에 결과를 표시합니다.

---

### ✅ 전체 코드

```python
import cv2
import numpy as np

# 마우스 클릭한 점들을 저장할 리스트
points = []

def mouse_callback(event, x, y, flags, param):
    global points, img_copy

    if event == cv2.EVENT_LBUTTONDOWN and len(points) < 4:
        points.append((x, y))
        # 점 찍기
        cv2.circle(img_copy, (x, y), 5, (0, 255, 0), -1)
        if len(points) > 1:
            cv2.line(img_copy, points[-2], points[-1], (255, 0, 0), 2)
        # 마지막 점일 경우 첫점과 연결
        if len(points) == 4:
            cv2.line(img_copy, points[3], points[0], (255, 0, 0), 2)

def perspective_projection_transform(img):
    global points, img_copy

    h, w = img.shape[:2]
    # 왼쪽 원본 + 오른쪽 복사본을 옆으로 붙이기
    display_img = np.hstack((img.copy(), img.copy()))
    img_copy = display_img.copy()

    cv2.namedWindow("Perspective Transform")
    cv2.setMouseCallback("Perspective Transform", mouse_callback)

    while True:
        cv2.imshow("Perspective Transform", img_copy)

        key = cv2.waitKey(1) & 0xFF

        # ESC 키로 종료
        if key == 27:
            break

        # 4점 선택되면 변환 수행
        if len(points) == 4:
            # 변환 대상 점 (사각형)
            dst_pts = np.float32([
                [0, 0],
                [w - 1, 0],
                [w - 1, h - 1],
                [0, h - 1]
            ])

            # perspective transform
            M = cv2.getPerspectiveTransform(np.float32(points), dst_pts)
            warped = cv2.warpPerspective(img, M, (w, h))

            # 오른쪽 영역에 결과 붙이기
            img_copy[:, w:] = warped

            # 변환 완료 후 다시 클릭 못하게 막기
            points.clear()

    cv2.destroyAllWindows()

# 예시 실행
# img = cv2.imread('social-media_640.jpg')
# perspective_projection_transform(img)
```

---

### 📝 사용법

1. 이미지를 불러오고 `perspective_projection_transform(img)`에 넘겨 실행하세요.
2. 왼쪽 이미지에서 **시계 방향 또는 반시계 방향으로 4점**을 클릭하세요.
3. 사각형이 자동으로 그려지고, 오른쪽 이미지에 투시 변환 결과가 나타납니다.
4. `ESC` 키를 누르면 창이 닫힙니다.

---

필요 시, 클릭한 좌표 출력이나 여러번 다시 시도 기능도 추가할 수 있습니다. 원하시나요?


</details>

<details>
<summary>📝 질문과 답 2 (오류 수정) - claude 3.5</summary>

### 질문 
---
문제 1번에 왼쪽 위부터 반시계 방향으로 하면 정상적으로 보여주도록 해줘 지금 순서가 반대로 대어 있어


### 답변
---
네, 현재 코드에서는 왼쪽 위부터 반시계 방향으로 클릭하도록 되어있지만, 실제 변환 시점에서 순서가 반대로 되어있네요. `perspective_projection_transform` 함수를 수정하겠습니다.

죄송합니다. 다시 한번 시도해보겠습니다.

죄송합니다. 파일 수정이 제대로 적용되지 않았네요. 다시 한번 시도해보겠습니다.

죄송합니다. Jupyter Notebook 파일의 수정이 예상보다 어려운 것 같습니다. 현재 코드의 문제점과 해결 방법을 설명드리겠습니다:

현재 코드에서는 `dst_pts`가 다음과 같이 정의되어 있습니다:
```python
dst_pts = np.float32([
    [0, 0],          # 왼쪽 위
    [w - 1, 0],      # 오른쪽 위
    [w - 1, h - 1],  # 오른쪽 아래
    [0, h - 1]       # 왼쪽 아래
])
```

이를 다음과 같이 수정해야 합니다:
```python
dst_pts = np.float32([
    [0, 0],          # 왼쪽 위
    [0, h - 1],      # 왼쪽 아래
    [w - 1, h - 1],  # 오른쪽 아래
    [w - 1, 0]       # 오른쪽 위
])
```

이렇게 수정하면:
1. 왼쪽 위 (0, 0)에서 시작
2. 왼쪽 아래 (0, h-1)로 이동
3. 오른쪽 아래 (w-1, h-1)로 이동
4. 오른쪽 위 (w-1, 0)로 이동

이렇게 반시계 방향으로 순서가 맞춰지게 됩니다.

Jupyter Notebook 파일을 직접 수정하시려면:
1. 노트북을 열어서 해당 셀을 찾으세요
2. `dst_pts` 배열의 순서를 위와 같이 수정하세요
3. 셀을 실행하시면 됩니다

이렇게 하면 왼쪽 위부터 반시계 방향으로 클릭했을 때 정상적으로 변환이 이루어질 것입니다.

</details>

<details>
<summary>📝 질문과 답 3 (np.hstack) - chat gpt 4o</summary>

### 질문
---
np.hstack((img.copy(), img.copy())) 은 왜 사용한거야?



### 답변
---

좋은 질문입니다!
`np.hstack((img.copy(), img.copy()))` 의 의미와 **이유**는 아래와 같습니다:

---

### 🔍 `np.hstack((img.copy(), img.copy()))` 의미

이 코드는:

```python
np.hstack((img.copy(), img.copy()))
```

* `img.copy()`로 **입력 이미지를 복사**한 것을 두 번 준비하고,
* `np.hstack(...)`을 이용해 **좌우로 나란히(horizontal stack)** 붙이는 것입니다.

즉, 하나는 왼쪽에, 하나는 오른쪽에 나타나게 됩니다.

---

### 🧠 왜 이렇게 했을까?

#### ✅ 이유 1: **왼쪽은 원본**, 오른쪽은 **결과 표시용**

* **왼쪽 이미지**: 사용자가 4점을 클릭할 입력용 원본
* **오른쪽 이미지**: 투시 변환 결과(perspective transform result)를 표시하기 위한 공간

#### ✅ 이유 2: **하나의 윈도우에 두 이미지를 함께 보여주기 위함**

* 하나의 OpenCV 창에서 `imshow`로 보여줄 수 있는 이미지는 1개뿐입니다.
* 그래서 두 이미지를 붙여서 하나로 만든 후 표시해야 합니다.

---

### 📌 예시: 입력 이미지가 `640x480`이면?

```python
img.shape = (480, 640, 3)
display_img = np.hstack((img.copy(), img.copy()))
```

➡ `display_img.shape = (480, 1280, 3)`
즉, 가로로 두 배 넓어진 이미지가 생성됩니다.

---

### ✅ 요약

| 코드                      | 설명                                   |
| ----------------------- | ------------------------------------ |
| `img.copy()`            | 원본 이미지 복사                            |
| `np.hstack((..., ...))` | 복사본 둘을 **가로로 붙임**                    |
| 목적                      | 한 창에 **입력 이미지 + 결과 이미지** 나란히 보여주기 위함 |

---

필요하다면 `np.vstack`을 써서 세로로 붙일 수도 있어요.
더 궁금한 점 있으신가요? 😊

---

</details>

<details>
<summary>📝 질문과 답 4 (클릭 저장 작업) - chat gpt 4o</summary>

### 질문 
---
코드에서 내가 클릭을 하면 4개의 좌표를 저장하는 작업을 어디서 하는거야?


### 답변
---
당신이 작성한 코드에서 **마우스로 클릭한 4개의 좌표를 저장하는 작업**은 함수 `mouse_callback()` 안에서 이루어집니다.
그 함수는 `cv2.setMouseCallback()`을 통해 등록됩니다.

---

### 🔍 클릭 좌표 저장 위치

```python
cv2.setMouseCallback("Perspective Transform", mouse_callback)
```

⬆️ 이 줄이 핵심입니다.
OpenCV는 `"Perspective Transform"`이라는 윈도우에서 **마우스 이벤트가 발생할 때마다** `mouse_callback` 함수를 호출하게 됩니다.

---

### 🔧 `mouse_callback()` 함수의 역할

이 함수는 이렇게 정의되어 있어야 합니다 (앞에서 전체 코드 참고):

```python
points = []  # 전역 리스트

def mouse_callback(event, x, y, flags, param):
    global points, img_copy

    if event == cv2.EVENT_LBUTTONDOWN and len(points) < 4:
        points.append((x, y))  # ← 여기서 좌표를 저장합니다
        cv2.circle(img_copy, (x, y), 5, (0, 255, 0), -1)
        if len(points) > 1:
            cv2.line(img_copy, points[-2], points[-1], (255, 0, 0), 2)
        if len(points) == 4:
            cv2.line(img_copy, points[3], points[0], (255, 0, 0), 2)
```

---

### ✅ 정리하면:

| 부분                          | 설명                                 |
| --------------------------- | ---------------------------------- |
| `cv2.setMouseCallback(...)` | 마우스 이벤트를 등록                        |
| `mouse_callback()` 함수       | 클릭 시 `(x, y)` 좌표를 `points` 리스트에 저장 |
| `points.append((x, y))`     | 좌표 저장하는 코드 (좌클릭 시)                 |

---

즉, **클릭 시 좌표 저장은 `mouse_callback` 함수에서 이루어지고**,
`perspective_projection_transform()` 함수에서는 `points`가 4개가 되면 변환을 수행합니다.

필요하다면 이 흐름을 시각적으로 설명해드릴 수도 있어요!


</details>

<details>
<summary>📝 질문과 답 1 (기초 구현)</summary>

### 질문 
---


### 답변
---

</details>

# 문제 2.

'beach_640.jpg' 파읽을 읽고 k-Means algorithm을 이용해서 5 개의 색만으로 벡터 양자화한 후에,
원 영상과 벡터 양자화된 영상을 나란히 표시하라.

다음 그림은 실행 결과 예이다.

![beach_cluster_5](beach_cluster_5.png)

In [25]:
import cv2
import numpy as np

def kmeans_color_quantization(img, K=5):
    Z = img.reshape((-1, 3))  # (H×W, 3)로 reshape
    Z = np.float32(Z)

    # K-means parameters
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    attempts = 10

    # k-means 알고리즘 적용
    ret, labels, centers = cv2.kmeans(Z, K, None, criteria, attempts, cv2.KMEANS_RANDOM_CENTERS)

    centers = np.uint8(centers)  # 색상은 정수여야 함
    quantized = centers[labels.flatten()]  # 각 픽셀에 클러스터 색상 적용
    quantized_img = quantized.reshape((img.shape))  # 다시 원본 크기로

    return quantized_img

def show_quantized_result():
    img = cv2.imread('beach_640.jpg')
    if img is None:
        print("이미지를 불러올 수 없습니다.")
        return

    quantized_img = kmeans_color_quantization(img, K=5)

    # 원본과 결과 나란히 표시
    combined = np.hstack((img, quantized_img))
    cv2.imshow("Original (Left) + Quantized (Right, K=5)", combined)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# 실행
show_quantized_result()


<details>
<summary>📝 코드 설명 (클릭하여 펼치기)</summary>

이러한 내용을 문제 2번 코드 밑에 열고 닫을 수 있도록 편의성있게 작성해줘

### 1. 함수 구조
```python
def kmeans_color_quantization(img, K=5):
    Z = img.reshape((-1, 3))  # (H×W, 3)로 reshape
    Z = np.float32(Z)
```
- 입력 이미지를 k-means 알고리즘에 적합한 형태로 변환
- 이미지 크기: H(높이) × W(너비) × 3(RGB 채널)
- 각 픽셀을 하나의 데이터 포인트로 변환

### 2. K-means 파라미터 설정
```python
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
attempts = 10
```
- 알고리즘 종료 조건 설정
  - 최대 10번 반복
  - 정확도 1.0 이하에서 종료
- 10번의 시도 중 최적의 결과 선택

### 3. K-means 알고리즘 적용
```python
ret, labels, centers = cv2.kmeans(Z, K, None, criteria, attempts, cv2.KMEANS_RANDOM_CENTERS)
```
- 입력 데이터를 K개의 클러스터로 군집화
- 반환값:
  - `ret`: 최적화된 목적 함수 값
  - `labels`: 각 픽셀의 클러스터 번호
  - `centers`: 각 클러스터의 대표 색상

### 4. 결과 이미지 생성
```python
centers = np.uint8(centers)  # 색상은 정수여야 함
quantized = centers[labels.flatten()]  # 각 픽셀에 클러스터 색상 적용
quantized_img = quantized.reshape((img.shape))  # 다시 원본 크기로
```
- 클러스터 중심점을 8비트 정수형으로 변환
- 각 픽셀을 해당 클러스터의 대표 색상으로 대체
- 원본 이미지 크기로 재구성

### 5. 결과 표시
```python
def show_quantized_result():
    # 이미지 로드 및 양자화
    # 원본과 결과를 나란히 표시
```
- 원본 이미지와 양자화된 이미지를 가로로 연결하여 표시
- 왼쪽: 원본 이미지
- 오른쪽: 5가지 색상으로 양자화된 이미지

### 주요 목적
1. 이미지의 모든 픽셀을 5개의 대표 색상으로 군집화
2. 각 픽셀을 가장 가까운 대표 색상으로 대체
3. 이미지의 색상 수를 줄이면서 주요 색상 정보 유지

### 활용 분야
- 이미지 압축
- 스타일 변환
- 이미지 분석
- 데이터 시각화

</details>

<details>
<summary>📝 질문과 답 (K-means가 이미지 색상 축소(양자화)에 효과적인가?)</summary>

### 질문 
---
이미지의 색상을 줄이는 데 왜 K-means가 적합한가요? 다른 방법과 비교했을 때 어떤 이점이 있나요?


### 답변
---
K-means는 다차원 공간(여기서는 RGB 색상 공간)에서 비슷한 색상들을 하나의 클러스터로 묶는 알고리즘입니다.
이미지에는 수만 가지 색상이 있을 수 있지만, 실제로는 시각적으로 유사한 색상들이 많습니다.
K-means는 그런 색상들을 효과적으로 묶고, 그 중심값을 대표 색으로 사용함으로써 시각적 품질을 크게 해치지 않으면서도 색상 수를 줄일 수 있습니다.

✅ 대체 방법과 비교:

단순 색상 양자화 (e.g., RGB를 64단계로 줄이기): 경계가 명확해져 품질이 떨어질 수 있음.

Median Cut, Octree Quantization 등도 존재하지만, K-means는 유클리디안 거리 기반의 최적화된 색 분할을 제공한다는 점에서 강력합니다.`

</details>

<details>
<summary>📝 질문과 답 (왜 np.float32로 변환하나요?)</summary>

### 질문 
---
왜 이미지 데이터를 float32로 바꾸는 게 중요할까요? 그냥 정수형으로 해도 되지 않나요?


### 답변
---
OpenCV의 cv2.kmeans()는 클러스터 중심 계산과 거리 계산을 부동소수점으로 수행해야 합니다.
정수형(uint8)으로 연산하면 거리 계산 정확도가 낮아지거나 오버플로우/언더플로우 문제가 발생할 수 있습니다.

또한, cv2.kmeans()는 내부적으로 OpenCV의 C++ API를 호출하는데, 이 API는 float32 형식의 입력을 기대합니다.
정수형으로 제공할 경우 예외가 발생하거나 잘못된 결과를 낼 수 있습니다.

</details>

<details>
<summary>📝 질문과 답 (왜 k-means의 결과가 항상 같지 않을 수 있나요?)</summary>

### 질문 
---
같은 K 값으로 여러 번 실행했을 때 결과가 다를 수 있는데, 왜 그런가요?


### 답변
---
K-means는 랜덤 초기 중심점(random init) 으로 시작하므로, 서로 다른 결과가 나올 수 있습니다.
이를 보완하기 위해 attempts=10처럼 여러 번 초기화하고 그 중 최적의 결과를 선택하도록 설정합니다.

➡️ K-means는 로컬 최적해(local minima)에 빠지기 쉬운 알고리즘입니다.
그래서 cv2.KMEANS_RANDOM_CENTERS는 결과 품질에 중요한 영향을 줍니다.



</details>

<details>
<summary>📝 질문과 답 (색상 공간을 바꾸는 것이 성능에 어떤 영향을 줄 수 있을까?)</summary>

### 질문 
---
RGB가 아닌 다른 색상 공간(HSV, Lab 등)을 사용하면 결과에 어떤 차이가 생길까요?


### 답변
---
RGB는 사람의 시각적 인식과는 달리 선형적인 색상 공간입니다.
즉, RGB에서의 거리 계산은 실제 색상 인식 차이와 다를 수 있습니다.

CIELab 공간은 사람이 인식하는 색상 차이를 더 잘 반영합니다.

HSV는 색조(Hue)와 채도(Saturation), 명도(Value)를 분리하여 조절이 쉬움.

➡️ 색상 공간을 Lab으로 변환한 후 K-means를 적용하면 더 시각적으로 일관된 양자화 결과를 얻을 수 있습니다.

</details>

# 문제 3.

문제 2의 프로그램을 다음과 같이 수정하라.

- OpenCV의 trackBar interface를 이용해서 k-Means의 cluster 개수를 조정하면서 벡터 양자화된 영상을 윈도우에 표시하라. 단, cluster 개수의 최솟값은 1로 설정하고, cluster의 개수가 1인 경우에는 원 영상을 표시하라. 따라서 실제 cluster의 최솟값은 2가 된다.


다음 그림은 실행 결과 예이다.

![beach_cluster_tarckbar](beach_cluster_tarckbar.png)

In [4]:
import cv2
import numpy as np

def kmeans_color_quantization(img, K):
    Z = img.reshape((-1, 3))
    Z = np.float32(Z)

    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    attempts = 10

    ret, labels, centers = cv2.kmeans(Z, K, None, criteria, attempts, cv2.KMEANS_RANDOM_CENTERS)
    centers = np.uint8(centers)
    quantized = centers[labels.flatten()]
    quantized_img = quantized.reshape((img.shape))

    return quantized_img

def on_trackbar(val):
    global original_img

    if val == 0:
        combined = np.hstack((original_img, original_img))
    else:
        k = val + 1  # val=1이면 k=2
        quantized_img = kmeans_color_quantization(original_img, k)
        combined = np.hstack((original_img, quantized_img))

    cv2.imshow("KMeans Quantization", combined)

def run_kmeans_with_trackbar():
    global original_img

    original_img = cv2.imread('beach_640.jpg')
    if original_img is None:
        print("이미지를 불러올 수 없습니다.")
        return

    window_name = "KMeans Quantization"
    cv2.namedWindow(window_name)
    cv2.createTrackbar("K (0:원본, 1~18 → K=2~20)", window_name, 0, 18, on_trackbar)

    on_trackbar(0)

    while True:
        # 창이 닫혔는지 확인
        if cv2.getWindowProperty(window_name, cv2.WND_PROP_VISIBLE) < 1:
            break
        if cv2.waitKey(1) & 0xFF == 27:  # ESC 키로 종료
            break

    cv2.destroyAllWindows()

# 실행
run_kmeans_with_trackbar()


<details>
<summary>📝 질문과 답 1 (기초 구현) - chat gpt 4o</summary>

### 질문 
---
문제에 맞게 구현해줘


### 답변
---
아래는 **문제 3**에서 요구한 대로 `OpenCV의 trackBar`를 이용하여 **클러스터 수(K)** 를 조정하면서 실시간으로 `k-Means` 벡터 양자화를 수행하는 전체 코드입니다.

---

### ✅ 핵심 기능 요약

* `cv2.createTrackbar`를 이용해 클러스터 개수를 조정
* `K=1`일 때는 **원본 이미지 그대로 표시**
* `K≥2`일 때는 **k-Means 벡터 양자화된 이미지 표시**
* 결과는 하나의 OpenCV 창에서 **실시간 갱신**

---

### 🧪 전체 코드

```python
import cv2
import numpy as np

def kmeans_color_quantization(img, K):
    Z = img.reshape((-1, 3))
    Z = np.float32(Z)

    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    attempts = 10

    ret, labels, centers = cv2.kmeans(Z, K, None, criteria, attempts, cv2.KMEANS_RANDOM_CENTERS)
    centers = np.uint8(centers)
    quantized = centers[labels.flatten()]
    quantized_img = quantized.reshape((img.shape))

    return quantized_img

def on_trackbar(val):
    global original_img

    k = cv2.getTrackbarPos("K (min 1)", "KMeans Quantization")

    # 최소 클러스터 1: 원본 그대로
    if k <= 1:
        combined = np.hstack((original_img, original_img))
    else:
        quantized_img = kmeans_color_quantization(original_img, k)
        combined = np.hstack((original_img, quantized_img))

    cv2.imshow("KMeans Quantization", combined)

def run_kmeans_with_trackbar():
    global original_img

    original_img = cv2.imread('beach_640.jpg')
    if original_img is None:
        print("이미지를 불러올 수 없습니다.")
        return

    # 창 생성 및 트랙바 설정
    cv2.namedWindow("KMeans Quantization")
    cv2.createTrackbar("K (min 1)", "KMeans Quantization", 1, 10, on_trackbar)

    # 초기 표시
    on_trackbar(1)

    while True:
        key = cv2.waitKey(1)
        if key == 27:  # ESC
            break

    cv2.destroyAllWindows()

# 실행
# run_kmeans_with_trackbar()
```

---

### 📌 실행 방법

1. `beach_640.jpg` 파일이 같은 폴더에 있어야 합니다.
2. 마지막 줄 `run_kmeans_with_trackbar()`를 호출하면 실행됩니다.
3. 윈도우의 트랙바를 움직여서 K 값을 1\~10 범위로 조정하세요.

   * `K = 1` → 원본
   * `K ≥ 2` → kMeans 벡터 양자화 이미지

---

### ✅ 추가 팁

* 트랙바 최대값은 `10`으로 설정했지만, 원하면 `20`, `50` 등으로 확장 가능합니다.
* 성능을 위해 이미지 크기를 줄여 처리하거나 `cv2.resize()`를 써도 좋습니다.

필요하면 저장 기능, 속도 개선, UI 꾸미기도 추가해 드릴 수 있어요.


</details>

<details>
<summary>📝 질문과 답 2 (기능 구현) - chat gpt 4o</summary>

### 질문 
---
최대 20개로 해주고 트랙바에 0은 삭제 부탁해
---


### 답변
---
물론입니다! 다음은 다음 조건을 모두 반영한 코드입니다:

---

### ✅ 수정 사항

* ✅ 클러스터 개수(K)의 **최대값은 20**
* ✅ 트랙바의 **최소값은 2** (1 또는 0은 제거)
* ✅ 트랙바 이름은 `"K (2~20)"`

---

### 🧪 최종 코드

```python
import cv2
import numpy as np

def kmeans_color_quantization(img, K):
    Z = img.reshape((-1, 3))
    Z = np.float32(Z)

    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    attempts = 10

    ret, labels, centers = cv2.kmeans(Z, K, None, criteria, attempts, cv2.KMEANS_RANDOM_CENTERS)
    centers = np.uint8(centers)
    quantized = centers[labels.flatten()]
    quantized_img = quantized.reshape((img.shape))

    return quantized_img

def on_trackbar(val):
    global original_img

    # 트랙바 값에 +2 해서 실제 K 값으로 사용
    k = val + 2

    quantized_img = kmeans_color_quantization(original_img, k)
    combined = np.hstack((original_img, quantized_img))
    cv2.imshow("KMeans Quantization", combined)

def run_kmeans_with_trackbar():
    global original_img

    original_img = cv2.imread('beach_640.jpg')
    if original_img is None:
        print("이미지를 불러올 수 없습니다.")
        return

    # 창 및 트랙바 생성
    cv2.namedWindow("KMeans Quantization")
    cv2.createTrackbar("K (2~20)", "KMeans Quantization", 0, 18, on_trackbar)
    # 0~18 이므로 실제 값은 2~20

    # 초기 표시 (K=2)
    on_trackbar(0)

    while True:
        if cv2.waitKey(1) & 0xFF == 27:  # ESC 키로 종료
            break

    cv2.destroyAllWindows()

# 실행
# run_kmeans_with_trackbar()
```

---

### 🔧 동작 요약

* 트랙바 값 범위는 `0~18`, 실제 K는 `val + 2` → `2~20`
* 처음 실행 시 `K = 2`로 초기 표시
* 원본과 양자화 이미지를 **좌우로 나란히** 보여줌

---

원하시면 "K 값 표시 텍스트"나 이미지 저장 버튼도 추가해 드릴 수 있어요. 😊


</details>

<details>
<summary>📝 질문과 답 (왜 색상 양자화에 K-means 알고리즘이 적합한가요?)</summary>

### 질문 
---
이미지의 색상을 줄이려면 다른 방법도 있을 텐데, 왜 굳이 K-means를 사용하는 걸까요?


### 답변
---
이미지에는 수천~수만 가지 색상이 포함되지만, 많은 색상들이 서로 비슷합니다. K-means는 이런 비슷한 색들을 군집화(clustering) 하여 대표 색상 중심(centroid) 으로 묶어줍니다.

이 방식은 다음과 같은 이점을 가집니다:

색상 수를 줄이면서도 시각적으로 유사한 품질을 유지

압축률 향상

엣지 검출, 물체 분할 등 후속 처리 전 단계로 유용

</details>